# M6 NO FAULT

IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M6\NO_FAULT6


SETTAGGIO DELLA VARIABILE RELATIVA AL GUASTO

In [3]:
guasto=0

## RCOU
VARIABILE RCOU POTENZA MOTORI.

Andiamo a prendere una potenza media di 1350 per considerare il momento il cui inizia/finisce il volo

In [4]:
rcou_m6_nofault = pd.read_csv("RCOU.csv")
rcou_m6_nofault = rcou_m6_nofault.astype("float64")
rcou_m6_nofault = rcou_m6_nofault.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m6_nofault = rcou_m6_nofault[((rcou_m6_nofault['C9'] >= potenza) & (rcou_m6_nofault['C10'] >= potenza) & (rcou_m6_nofault['C11'] >= potenza) & (rcou_m6_nofault['C12'] >= potenza) & (rcou_m6_nofault['C13'] >= potenza) & (rcou_m6_nofault['C14']>= potenza))]
rcou_m6_nofault=rcou_m6_nofault.reset_index(drop=True)
display(rcou_m6_nofault)

,TimeUS,C9,C10,C11,C12,C13,C14
0,73585436.0,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0
1,73609813.0,1415.0,1428.0,1402.0,1439.0,1470.0,1366.0
2,73620421.0,1415.0,1438.0,1423.0,1431.0,1483.0,1364.0
3,73663445.0,1448.0,1452.0,1468.0,1431.0,1530.0,1356.0
4,73673952.0,1451.0,1461.0,1462.0,1449.0,1523.0,1379.0
...,...,...,...,...,...,...,...
5481,141114087.0,1562.0,1440.0,1526.0,1481.0,1478.0,1529.0
5482,141125072.0,1574.0,1422.0,1525.0,1479.0,1490.0,1516.0
5483,141136462.0,1603.0,1378.0,1547.0,1451.0,1544.0,1454.0
5484,141345328.0,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0


Andiamo a prendere i tempi di inizio e di fine, così da poter effettuare il trimming delle altre tabelle

In [5]:
min=rcou_m6_nofault['TimeUS'][0]
max=rcou_m6_nofault['TimeUS'][len(rcou_m6_nofault)-1]
print(max)
print(min)

141376898.0
73585436.0


## ATT & XKF
### XKF
XKF utilizzato per fare poi il confronto con i valori di roll, pitch e yaw presenti att

In [6]:
xkf1_m6_nofault = pd.read_csv("XKF1_0.csv")
xkf1_m6_nofault = xkf1_m6_nofault.astype("float64")
xkf1_m6_nofault = xkf1_m6_nofault.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m6_nofault = xkf1_m6_nofault[((xkf1_m6_nofault['TimeUS'] >= min) & (xkf1_m6_nofault['TimeUS'] <= max	))]
xkf1_m6_nofault = xkf1_m6_nofault.reset_index(drop=True)
print(xkf1_m6_nofault)

           TimeUS  Roll  Pitch    Yaw
0      73597012.0  1.77  -1.55  15.93
1      73607972.0  1.70  -1.53  15.96
2      73617984.0  1.62  -1.51  15.99
3      73628863.0  1.62  -1.52  16.00
4      73638809.0  1.63  -1.51  16.01
...           ...   ...    ...    ...
6139  141325367.0 -0.09  -2.57   4.16
6140  141343293.0 -0.16  -2.48   3.98
6141  141353420.0 -0.13  -2.25   3.91
6142  141364373.0 -0.15  -2.06   3.89
6143  141374850.0 -0.24  -1.93   3.86

[6144 rows x 4 columns]


### ATT
Andiamo a prendere dal csv relativo ad ATT i dati relativi alle variabili di roll, pitch e yaw

In [7]:
att_m6_nofault = pd.read_csv("ATT.csv")
att_m6_nofault = att_m6_nofault.astype("float64")
att_m6_nofault = att_m6_nofault.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m6_nofault = att_m6_nofault[((att_m6_nofault['TimeUS'] >= min) & (att_m6_nofault['TimeUS'] <= max	))]
att_m6_nofault=att_m6_nofault.reset_index(drop=True)
print(att_m6_nofault)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      73598303.0     0.00  1.77     -0.55  -1.55   18.84  15.93
1      73609051.0     0.00  1.70     -0.54  -1.53   18.84  15.96
2      73619815.0     0.00  1.62     -0.53  -1.51   18.84  15.99
3      73630059.0     0.01  1.62     -0.53  -1.52   18.84  16.00
4      73640504.0     0.01  1.63     -0.53  -1.51   18.84  16.01
...           ...      ...   ...       ...    ...     ...    ...
6139  141327089.0    -0.61 -0.09     -2.50  -2.57    7.54   4.16
6140  141344718.0    -0.43 -0.16     -2.64  -2.48    7.54   3.98
6141  141354891.0    -0.25 -0.13     -2.76  -2.25    7.54   3.91
6142  141365784.0    -0.05 -0.15     -2.88  -2.06    7.54   3.89
6143  141376080.0     0.13 -0.24     -2.98  -1.93    7.54   3.86

[6144 rows x 7 columns]


Tramite un ciclo for si va a vedere se i valori di XKF corrispondono con quelli di ATT in caso contrario fa una media tra i due

In [8]:
from numpy import empty

if len(att_m6_nofault)>len(xkf1_m6_nofault):
    to_add=att_m6_nofault[len(xkf1_m6_nofault):]
    att_m6_nofault=att_m6_nofault[:len(xkf1_m6_nofault)]

if len(xkf1_m6_nofault)>len(att_m6_nofault):
    to_add=xkf1_m6_nofault[len(att_m6_nofault):]

for idx,i in enumerate(att_m6_nofault['Roll']):
    if(xkf1_m6_nofault['Roll'][idx] != i):
        att_m6_nofault['Roll'][idx] = ((att_m6_nofault['Roll'][idx] + xkf1_m6_nofault['Roll'][idx])/2)

for idx,i in enumerate(att_m6_nofault['Pitch']):
    if(xkf1_m6_nofault['Pitch'][idx] != i):
        att_m6_nofault['Pitch'][idx] = ((att_m6_nofault['Pitch'][idx] + xkf1_m6_nofault['Pitch'][idx])/2)

for idx,i in enumerate(att_m6_nofault['Yaw']):
    if(xkf1_m6_nofault['Yaw'][idx] != i):
        att_m6_nofault['Yaw'][idx] = ((att_m6_nofault['Yaw'][idx] + xkf1_m6_nofault['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m6_nofault=pd.concat([att_m6_nofault,to_add])

print(att_m6_nofault)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      73598303.0     0.00  1.77     -0.55  -1.55   18.84  15.93
1      73609051.0     0.00  1.70     -0.54  -1.53   18.84  15.96
2      73619815.0     0.00  1.62     -0.53  -1.51   18.84  15.99
3      73630059.0     0.01  1.62     -0.53  -1.52   18.84  16.00
4      73640504.0     0.01  1.63     -0.53  -1.51   18.84  16.01
...           ...      ...   ...       ...    ...     ...    ...
6139  141327089.0    -0.61 -0.09     -2.50  -2.57    7.54   4.16
6140  141344718.0    -0.43 -0.16     -2.64  -2.48    7.54   3.98
6141  141354891.0    -0.25 -0.13     -2.76  -2.25    7.54   3.91
6142  141365784.0    -0.05 -0.15     -2.88  -2.06    7.54   3.89
6143  141376080.0     0.13 -0.24     -2.98  -1.93    7.54   3.86

[6144 rows x 7 columns]


## IMU
VALORI DI ACCELERAZIONE E DEL GIROSCOPIO

Per le 3 tabelle di IMU si sono presi i soli valori relativi a GyrX, GyrY, GyrZ, AccX, AccY, AccZ.
In seguito si è poi creata una singola tabella popolandola con la media dei valori delle tre tabelle.

In [10]:
imu_0_m6_nofault = pd.read_csv("IMU_0.csv")
imu_0_m6_nofault = imu_0_m6_nofault.astype("float64")
imu_0_m6_nofault = imu_0_m6_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m6_nofault = imu_0_m6_nofault[((imu_0_m6_nofault['TimeUS'] >= min) & (imu_0_m6_nofault['TimeUS'] <= max))]

imu_1_m6_nofault = pd.read_csv("IMU_1.csv")
imu_1_m6_nofault = imu_1_m6_nofault.astype("float64")
imu_1_m6_nofault = imu_1_m6_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m6_nofault = imu_1_m6_nofault[((imu_1_m6_nofault['TimeUS'] >= min) & (imu_1_m6_nofault['TimeUS'] <= max))]

imu_2_m6_nofault = pd.read_csv("IMU_2.csv")
imu_2_m6_nofault = imu_2_m6_nofault.astype("float64")
imu_2_m6_nofault = imu_2_m6_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m6_nofault = imu_2_m6_nofault[((imu_2_m6_nofault['TimeUS'] >= min) & (imu_2_m6_nofault['TimeUS'] <= max))]

imu_m6_nofault = pd.concat((imu_0_m6_nofault, imu_1_m6_nofault, imu_2_m6_nofault))
imu_m6_nofault=imu_m6_nofault.groupby(imu_m6_nofault.TimeUS, as_index=False).mean()

display(imu_m6_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,73587431.0,0.147274,-0.138449,-0.001893,-0.119606,0.091852,-9.635330
1,73590478.0,0.092451,-0.148777,0.007550,-0.255108,-0.065997,-9.740719
2,73592092.0,-0.116242,0.046451,0.039096,-0.387778,-0.080363,-9.773333
3,73598199.0,0.136942,-0.077143,-0.004878,-0.089322,0.035817,-9.557465
4,73604174.0,-0.115818,0.011950,0.043407,-0.420610,-0.265488,-9.875872
...,...,...,...,...,...,...,...
23374,141365515.0,-0.036255,0.351672,-0.038613,-1.075691,-0.784255,-9.579230
23375,141368756.0,-0.025718,0.381097,-0.022933,-1.067247,-0.849385,-9.791680
23376,141370810.0,-0.157157,0.354411,-0.023660,-1.086078,-0.746119,-9.873503
23377,141373732.0,-0.240294,0.277462,-0.019804,-1.017304,-0.481268,-10.041165


## UNIONE DELLE TABELLE
Vengono unite le tabelle relative a IMU, ATT e RCOU, riempiendo i valori mancanti con quelli vicini

In [11]:
m6_nofault = pd.merge_ordered(imu_m6_nofault,att_m6_nofault)
m6_nofault=m6_nofault.fillna(method="ffill").fillna(method="bfill")
m6_nofault = pd.merge_ordered(m6_nofault, rcou_m6_nofault)
m6_nofault=m6_nofault.fillna(method="ffill").fillna(method="bfill")

Vengono aggiornati i tempi facendo partire il volo all'istante 0, e viene inserita la colonna relativa al guasto, valorizzandola con la variabile *guasto*.

In [12]:
m6_nofault["TimeUS"] = m6_nofault["TimeUS"] - m6_nofault.iloc[0]["TimeUS"]
m6_nofault["TimeUS"] = m6_nofault["TimeUS"].astype(int)
m6_nofault["Guasto"] = guasto

In [ ]:
last_cell = m6_nofault.iloc[-1, m6_nofault.columns.get_loc('TimeUS')]
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)
df = pd.DataFrame(to_be, columns = ['TimeUS'])

Viene corretta la frequenza di campionamento in modo tale da avere 350 misurazioni per ogni secondo

In [ ]:
m6_nofault_final = pd.merge_ordered(m6_nofault,df)
m6_nofault_final=m6_nofault_final.fillna(method="ffill").fillna(method="bfill")
m6_nofault_final = m6_nofault_final[m6_nofault_final.TimeUS.isin(to_be)]

Viene formattata la colonna del tempo in maniera tale che sia più leggibile all'operatore umano

In [14]:
from datetime import timedelta
m6_nofault_final=m6_nofault_final.reset_index(drop=True)
m6_nofault_final['TimeUS'] = pd.to_datetime(m6_nofault_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m6_nofault_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.147274,-0.138449,-0.001893,-0.119606,0.091852,-9.635330,0.00,1.77,-0.55,-1.55,18.84,15.93,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
1,00:00:00.002857,0.147274,-0.138449,-0.001893,-0.119606,0.091852,-9.635330,0.00,1.77,-0.55,-1.55,18.84,15.93,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
2,00:00:00.005714,0.092451,-0.148777,0.007550,-0.255108,-0.065997,-9.740719,0.00,1.77,-0.55,-1.55,18.84,15.93,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
3,00:00:00.008571,-0.116242,0.046451,0.039096,-0.387778,-0.080363,-9.773333,0.00,1.77,-0.55,-1.55,18.84,15.93,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
4,00:00:00.011428,-0.116242,0.046451,0.039096,-0.387778,-0.080363,-9.773333,0.00,1.77,-0.55,-1.55,18.84,15.93,1377.0,1436.0,1362.0,1450.0,1416.0,1400.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23724,00:01:07.779468,-0.170500,0.261187,-0.083874,-1.089565,-1.073007,-9.274010,-0.25,-0.13,-2.76,-2.25,7.54,3.91,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0.0
23725,00:01:07.782325,-0.036255,0.351672,-0.038613,-1.075691,-0.784255,-9.579230,-0.05,-0.15,-2.88,-2.06,7.54,3.89,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0.0
23726,00:01:07.785182,-0.025718,0.381097,-0.022933,-1.067247,-0.849385,-9.791680,-0.05,-0.15,-2.88,-2.06,7.54,3.89,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0.0
23727,00:01:07.788039,-0.157157,0.354411,-0.023660,-1.086078,-0.746119,-9.873503,-0.05,-0.15,-2.88,-2.06,7.54,3.89,1405.0,1392.0,1399.0,1398.0,1360.0,1435.0,0.0


## Analisi nel tempo e in frequenza

Funzione che calcola il Root Mean Square

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

Viene definita la funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza


In [16]:
from numpy import mean, sqrt, square
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m6_nofault_final)/350)):
        V1=m6_nofault_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

Si richiama la funzione descritta sopra per ogni variabile in modo da avere le statistiche sia nel tempo che in frequenza

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')

merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_11232\2215761558.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_11232\2215761558.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.061274,0.005950,0.034892,0.098425,0.0,88.0,12.505537,2.495923,0.035329,0.003595,-0.155294,0.069522,96.0,95.0,2.565082,1.511218,0.179938,0.013575,-1.404852,0.214275,71.0,70.0,4.353594,1.677543,-0.049156,0.100520,-1.142779,0.320390,108.0,105.0,9.320561,6.488387,0.144892,0.076009,-0.734314,0.311105,112.0,118.0,19.383560,7.656605,-11.305159,0.796879,-0.819977,11.340248,0.0,107.0,145.797053,26.320956,0.488291,1.411511,-1.742137,1.282933,147.0,146.0,27.047619,11.343294,-0.750199,0.407411,-1.619929,0.984403,121.0,120.0,18.448338,2.928681,19.617436,12.686633,-1.229936,19.937259,137.0,136.0,14.878230,12.747360,1562.116809,6492.840602,-0.734511,1564.187734,99.0,97.0,672.524465,481.079478,1519.467236,1393.478209,0.037201,1519.924403,115.0,118.0,756.638127,550.117705,1555.398860,6818.183313,-0.823808,1557.582862,95.0,97.0,409.367689,360.789205,1525.829060,2190.427839,-0.568468,1526.544630,99.0,97.0,873.586648,491.705253,1541.760684,1731.451136,0.331673,1542.320500,99.0,103.0,943.650795,891.326967,1541.239316,5426.136850,0.396884,1542.993619,119.0,118.0,1548.860942,701.296568,0
1,1.0,-0.061302,0.005933,0.042842,0.098357,0.0,99.0,12.453780,2.371632,0.035380,0.003586,-0.147350,0.069482,94.0,93.0,2.531256,1.748557,0.180256,0.013572,-1.400410,0.214536,63.0,62.0,4.408981,1.622185,-0.047815,0.100866,-1.152215,0.320728,106.0,103.0,9.437702,7.075060,0.145429,0.075894,-0.735188,0.311173,119.0,125.0,20.764649,7.889268,-11.305087,0.794610,-0.813996,11.340077,0.0,99.0,146.685513,26.098293,0.483835,1.414477,-1.743160,1.282402,147.0,146.0,27.506157,11.298343,-0.747585,0.408656,-1.620545,0.983046,120.0,119.0,18.541344,3.288963,19.637670,12.794611,-1.230996,19.959869,146.0,145.0,15.270929,12.491215,1562.406250,6503.831642,-0.729737,1564.480311,102.0,100.0,681.352567,482.601703,1519.329545,1396.181689,0.016097,1519.787644,116.0,119.0,800.747977,553.635769,1555.752841,6842.864664,-0.821449,1557.944263,97.0,99.0,441.647772,374.175050,1525.610795,2200.956350,-0.583363,1526.329913,105.0,103.0,883.430741,600.899048,1541.843750,1728.947026,0.339982,1542.402731,101.0,99.0,924.681403,883.888640,1541.340909,5414.310800,0.408715,1543.091289,117.0,116.0,1547.521765,720.323207,0
2,2.0,-0.061041,0.005940,0.036872,0.098231,0.0,94.0,12.502659,2.343510,0.035068,0.003610,-0.169144,0.069498,93.0,92.0,2.366637,1.858441,0.180509,0.013556,-1.395499,0.214712,71.0,70.0,4.440444,1.541333,-0.045959,0.101797,-1.147513,0.321902,108.0,105.0,9.735167,7.766944,0.146023,0.075803,-0.737305,0.311306,120.0,126.0,21.276764,8.142415,-11.305089,0.792353,-0.807721,11.339980,0.0,107.0,147.538946,25.801408,0.479405,1.417386,-1.744131,1.281874,151.0,150.0,27.947076,11.245759,-0.744986,0.409880,-1.621156,0.981694,117.0,116.0,18.625490,3.638163,19.657790,12.901160,-

## Output
Salviamo la tabella ottenuta in un file csv

In [18]:
path_file = path_file.replace(r"M6\NO_FAULT6", "")
os.chdir(path_file)
df_merged.to_csv('m6_nofault.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
